# Querying Agents

## Imports

In [2]:
# AutoGen




# LangChain




## AutoGen

### Define helper methods

### Define the Agent

### Test the Agent

## LangChain

### Define helper methods

### Define the Agent

### Test the Agent